In [23]:
import os

import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import boto3
import mlflow
from mlflow.models import infer_signature

In [24]:
mlflow.set_tracking_uri("http://127.0.0.1:8000")

In [25]:
mlflow.create_experiment(name="change_mlflow_artifact_registry")
mlflow.set_experiment("change_mlflow_artifact_registry")


RestException: RESOURCE_ALREADY_EXISTS: Experiment 'change_mlflow_artifact_registry' already exists.

In [ ]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)

c:\Users\raksh\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris-model",
        signature=signature,
        input_example=X_train,
        registered_model_name="basic_lr_iris_model",
    )

Successfully registered model 'basic_lr_iris_model'.
2024/12/08 20:47:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: basic_lr_iris_model, version 1


🏃 View run fun-stag-889 at: http://127.0.0.1:8000/#/experiments/436638021526534897/runs/81c0298ea1574659b66097fd75da722f
🧪 View experiment at: http://127.0.0.1:8000/#/experiments/436638021526534897


Created version '1' of model 'basic_lr_iris_model'.


In [ ]:
import os

run_id = "81c0298ea1574659b66097fd75da722f"
base_path = "./mlruns"

# Walk through each experiment folder
for experiment in os.listdir(base_path):
    experiment_path = os.path.join(base_path, experiment)
    if os.path.isdir(experiment_path):
        if run_id in os.listdir(experiment_path):
            print(f"Run {run_id} found in experiment {experiment}")
            print("Contents of the run folder:")
            print(os.listdir(os.path.join(experiment_path, run_id)))


In [ ]:
print("Model URI:", model_info.model_uri)
print("Tracking URI:", mlflow.get_tracking_uri())

import os
print(os.listdir("./mlruns"))



Model URI: runs:/81c0298ea1574659b66097fd75da722f/iris-model
Tracking URI: http://127.0.0.1:8000
['.trash', '0', '166941913289191557', '207669226896184824', '567373898312505361', 'models']


In [ ]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

# Convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test, columns=iris_feature_names)

# Add the actual classes to the DataFrame
result["actual_class"] = y_test

# Add the model predictions to the DataFrame
result["predicted_class"] = predictions

result[:4]

NoSuchKey: An error occurred (NoSuchKey) when calling the ListObjectsV2 operation: None